# Exploring the relationships between reported pain and electrical current
Participants were exposed to 2 x 200 stimuli (400 total) lasting 2 seconds each. 
The stimuli amplitude were random from a uniform distribution spanning 0-->current associated with 5/10 

Analysis:
1. threshold (current value $x$ at 1/10 pain)
2. equation to describe relationship
   1. exponential vs linear
3. Split relationship up (perhaps at 1/10 pain) and fit linear after that threshold
4. Difference in inflection point on x axis (is there habituation between trial 0-200 and 201-400)


In [1]:
import os
print(os.getcwd())

c:\Users\j-cha\OneDrive - UBC\Experimental Pain Studies\experiments\pain habituation and mapping\analysis


In [2]:
# load libraries
import numpy as np
from utils import fitting 
import pandas as pd
import matplotlib.pyplot as plt
import os
# from scipy import optimize
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pickle


In [4]:
# find folders with painmap data to be loaded in
datadir = "../data/painmap/"
direcs = os.listdir(datadir)


p_data = {}
for d in direcs:
    if "painmaps" in os.listdir(datadir + d) and len(os.listdir(datadir + d + "/painmaps")) > 0:
        p_data[d] = {"direc": os.path.join(datadir, d, "painmaps")}

# print the data files to screen to check
for p in p_data.keys():
    print(p_data[p]["direc"])
    print(os.listdir(p_data[p]["direc"]))



../data/painmap/p01\painmaps
['block1.csv', 'block2.csv', 'data.csv']
../data/painmap/p02\painmaps
['0.csv', '010.csv', '011.csv', '012.csv', '013.csv', '08.csv', '09.csv', 'data.csv']
../data/painmap/p03\painmaps
['block1.csv', 'block2.csv', 'block3.csv', 'data.csv']
../data/painmap/p04\painmaps
['block1.csv', 'block2.csv', 'data.csv']
../data/painmap/p05\painmaps
['0.csv', '05.csv', '06.csv', '07.csv', 'data.csv']
../data/painmap/p06\painmaps
['1.csv', '2.csv', 'fit_data.pkl']


In [5]:
# load data from a specific participant

for i, p in enumerate(p_data.keys()):
    datafiles = os.listdir(p_data[p]["direc"])
    if "data.csv" in datafiles:
        df = pd.read_csv(os.path.join(p_data[p]["direc"], "data.csv"), header=0, index_col=None) 
        p_data[p]["data"] = df
    else:
        # loop over datafiles and load them
        pain_current_data = []
        for f in datafiles:
            if f.endswith(".csv"):
                df = pd.read_csv(os.path.join(p_data[p]["direc"], f), header=0, index_col=None)
                pain_current_data.append(df)
        pain_current_data = pd.concat(pain_current_data, axis=0, ignore_index=True)
        pain_current_data = pain_current_data.fillna(0)  # NANs were registered if potentiometer was bottomed out (zeros)

        # compile into data and put in dict
        p_data[p]["data"] = pain_current_data

        # write the data as one congruent file back to directory for later use.
        pain_current_data.to_csv(os.path.join(p_data[p]["direc"], "data.csv"), index=False)

# Fit functions to the pain-current relationship
The relationship between pain-current looks approximately exponential. This will be the main priorty

Exponential: $y=a*e^{(-b*x)} + c$

It may be possible to fit a linear function in the latter part of the pain-current relationship. We will use the 1/10 threshold as the start point for the linear fit in the latter half of the data

In [6]:
pain_cutoff_for_linearfit = 1.5

# Fit data per participant
for i, p in enumerate(p_data.keys()):
    xdata = p_data[p]["data"]["mA"]
    ydata = p_data[p]["data"]["painreport"]
    xmin = min(p_data[p]["data"]["mA"])
    xmax = max(p_data[p]["data"]["mA"])
    xintervals = np.linspace(xmin, xmax, xdata.shape[0])

    # normalized x axis
    p_data[p]["data"]["mA_norm"] = (xdata - xmin) / (xmax - xmin)

    # Filter for data above pain cut off for linear fitting
    filtdat = p_data[p]["data"][
        p_data[p]["data"]["painreport"] >= pain_cutoff_for_linearfit
    ]

    # Exponential fit
    p_data[p]["efit"] = fitting.fit_exp(
        p_data[p]["data"]["mA"], p_data[p]["data"]["painreport"]
    )
    p_data[p]["efit_y"] = fitting.exponential(
        p_data[p]["data"]["mA"], *p_data[p]["efit"][0]
    )
    p_data[p]["data"]["efit_y"] = p_data[p]["efit_y"]
    p_data[p]["efit_r2"] = fitting.rsquared(
        p_data[p]["data"]["painreport"], p_data[p]["efit_y"]
    )

    # R2 exponential evaluated on the same filtered data as the linear fit
    p_data[p]["efit_r2"] = fitting.rsquared(
        filtdat["painreport"], fitting.exponential(filtdat["mA"], *p_data[p]["efit"][0])
    )

    # exponential fit on normalized x axis
    p_data[p]["efit_norm"] = fitting.fit_exp(
        p_data[p]["data"]["mA_norm"], p_data[p]["data"]["painreport"]
    )
    p_data[p]["efit_norm_y"] = fitting.exponential(
        p_data[p]["data"]["mA_norm"], *p_data[p]["efit_norm"][0]
    )
    p_data[p]["data"]["efit_norm_y"] = p_data[p]["efit_norm_y"]

    # R2 exponential evaluated on the same filtered data as the linear fit
    p_data[p]["efit_r2_norm"] = fitting.rsquared(
        filtdat["painreport"],
        fitting.exponential(filtdat["mA_norm"], *p_data[p]["efit_norm"][0]),
    )

    # Fit linear to unnormalized x axis
    p_data[p]["lfit"] = fitting.fit_lin(filtdat["mA"], filtdat["painreport"])
    p_data[p]["lfit_y"] = fitting.linear(filtdat["mA"], *p_data[p]["lfit"][0])

    p_data[p]["lfit_r2"] = fitting.rsquared(
        filtdat["painreport"], fitting.linear(filtdat["mA"], *p_data[p]["lfit"][0])
    )

    # Fit linear to normalized x axis
    p_data[p]["lfit_norm"] = fitting.fit_lin(filtdat["mA_norm"], filtdat["painreport"])
    p_data[p]["lfit_norm_y"] = fitting.linear(
        filtdat["mA_norm"], *p_data[p]["lfit_norm"][0]
    )
    p_data[p]["lfit_r2_norm"] = fitting.rsquared(
        filtdat["painreport"],
        fitting.linear(filtdat["mA_norm"], *p_data[p]["lfit_norm"][0]),
    )

    # calculate radius of curvate for the exponential (looking to find the inflexion point wher the radius is minimum)
    # # efit-dot and doubledot -> fist and second derivative of exponential fit
    p_data[p]["efit_yd"] = np.gradient(
        fitting.exponential(
            np.linspace(xmin, xmax, xdata.shape[0]), *p_data[p]["efit"][0]
        ),
    )
    p_data[p]["efit_ydd"] = np.gradient(
        p_data[p]["efit_yd"],
    )

    # # radius of curvature of the exponential fit (https://mathworld.wolfram.com/RadiusofCurvature.html)
    p_data[p]["radius"] = ((1 + p_data[p]["efit_yd"] ** 2) ** (3 / 2)) / np.abs(
        p_data[p]["efit_ydd"]
    )

    print(f"{p} Exp R^2: {p_data[p]['efit_r2']: .3f}")
    print(f"{p} Lin R^2: {p_data[p]['lfit_r2']: .3f}")

p01 Exp R^2:  0.527
p01 Lin R^2:  0.572
p02 Exp R^2:  0.440
p02 Lin R^2:  0.517
p03 Exp R^2:  0.138
p03 Lin R^2:  0.437
p04 Exp R^2:  0.550
p04 Lin R^2:  0.627
p05 Exp R^2:  0.700
p05 Lin R^2:  0.749
p06 Exp R^2:  0.677
p06 Lin R^2:  0.780


In [7]:
p="p02"
p_data[p]["x_at_1y"] = (
        -1
        * (np.log((1 - p_data[p]["efit"][0][2]) / p_data[p]["efit"][0][0]))
        / p_data[p]["efit"][0][1]
    )
p_data[p]["x_at_2y"] = (
        -1
        * (np.log((2 - p_data[p]["efit"][0][2]) / p_data[p]["efit"][0][0]))
        / p_data[p]["efit"][0][1]
    )
p_data[p]["x_at_3y"] = (
        -1
        * (np.log((3 - p_data[p]["efit"][0][2]) / p_data[p]["efit"][0][0]))
        / p_data[p]["efit"][0][1]
    )
p_data[p]["x_at_4y"] = (
        -1
        * (np.log((4 - p_data[p]["efit"][0][2]) / p_data[p]["efit"][0][0]))
        / p_data[p]["efit"][0][1]
    )
p_data[p]["x_at_5y"] = (
        -1
        * (np.log((5 - p_data[p]["efit"][0][2]) / p_data[p]["efit"][0][0]))
        / p_data[p]["efit"][0][1]
    )

print(f"mA for pain 1: {round(p_data[p]['x_at_1y'],2)}")
print(f"mA for pain 2: {round(p_data[p]['x_at_2y'],2)}")
print(f"mA for pain 3: {round(p_data[p]['x_at_3y'],2)}")
print(f"mA for pain 4: {round(p_data[p]['x_at_4y'],2)}")
print(f"mA for pain 5: {round(p_data[p]['x_at_5y'],2)}")

mA for pain 1: 2.6
mA for pain 2: 3.61
mA for pain 3: 4.27
mA for pain 4: 4.76
mA for pain 5: 5.16


In [8]:
# save fitting results into pickle file
savedir = "../data/processed/"
with open(os.path.join(savedir,"pain_current_fits.pkl"), "wb") as f:
    pickle.dump(p_data, f)

In [9]:
def model_eval_plots(xdata, ydata, fig, row, col):
    # build figure
    fig.add_trace(
        go.Scatter(
            x=xdata,
            y=ydata,
            mode="markers",
            name=p,
            marker=dict(size=10, color=px.colors.qualitative.Plotly[i]),
            marker_opacity=0.3,
            
        ),
        row=row,
        col=col,
    )
    fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="white", row=row, col=col)

In [11]:
# Examining model fits metrics

fig = make_subplots(
    rows=len(p_data.keys()),
    cols=2,
    subplot_titles=("Exponential", "Linear"),
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.05,
    x_title="Current (mA)",
    y_title="residuals / sigma",
)
for i, p in enumerate(p_data.keys()):
    # standard residuals vs predicted plot
    stand_res_e = p_data[p]["efit"][2]["fvec"]
    pred_e = p_data[p]["efit_y"]
    stand_res_l = p_data[p]["lfit"][2]["fvec"]
    pred_l = p_data[p]["lfit_y"]

    model_eval_plots(pred_e, stand_res_e, fig, row=i + 1, col=1)
    model_eval_plots(pred_l, stand_res_l, fig, row=i + 1, col=2)
    fig.update_layout(
        title="Standardized Residual vs. Predicted Values",
        font=dict(size=18),
        width=800,
        height=1000,
        template="plotly_dark",
    )

fig.show()

In [12]:
# Function to plot pain as function of current with fitting results
def render_pain_current_plot(
    xdata, ydata, expfitparams, linfitparams, p, fig, plot_lin=True, plot_exp=True
):
    # min and max x values
    xmin = min(xdata)
    xmax = max(xdata)

    # build figure
    fig.add_trace(
        go.Scatter(
            x=xdata,
            y=ydata,
            mode="markers",
            name=p,
            marker=dict(size=10, color=px.colors.qualitative.Plotly[i]),
            marker_opacity=0.3,
        )
    )

    if plot_exp:
        fig.add_trace(
            go.Scatter(
                x=np.linspace(xmin, xmax, xdata.shape[0]),
                y=fitting.exponential(
                    np.linspace(xmin, xmax, xdata.shape[0]), *expfitparams[0]
                ),
                mode="lines",
                name=p + "exp",
                line=dict(dash="solid", width=3, color=px.colors.qualitative.Plotly[i]),
            )
        )

    if plot_lin:
        fig.add_trace(
            go.Scatter(
                x=np.linspace(xmin, xmax, xdata.shape[0]),
                y=fitting.linear(
                    np.linspace(xmin, xmax, xdata.shape[0]), *linfitparams[0]
                ),
                mode="lines",
                name=p + "linear",
                line=dict(dash="solid", width=3, color=px.colors.qualitative.Plotly[i]),
            )
        )


def render_pain_current_subplot(
    xdata,
    ydata,
    expfitparams,
    linfitparams,
    p,
    fig,
    plot_lin=True,
    plot_exp=True,
    row=1,
    col=1,
):
    # min and max x values
    xmin = min(xdata)
    xmax = max(xdata)

    # build figure
    fig.add_trace(
        go.Scatter(
            x=xdata,
            y=ydata,
            mode="markers",
            name=p,
            marker=dict(size=10, color=px.colors.qualitative.Plotly[i]),
            marker_opacity=0.3,
        ),
        row=row,
        col=col,
    )

    if plot_exp:
        fig.add_trace(
            go.Scatter(
                x=np.linspace(xmin, xmax, xdata.shape[0]),
                y=fitting.exponential(
                    np.linspace(xmin, xmax, xdata.shape[0]), *expfitparams[0]
                ),
                mode="lines",
                name=p + "exp",
                line=dict(dash="solid", width=3, color=px.colors.qualitative.Plotly[i]),
            ),
            row=row,
            col=col,
        )

    if plot_lin:
        fig.add_trace(
            go.Scatter(
                x=np.linspace(xmin, xmax, xdata.shape[0]),
                y=fitting.linear(
                    np.linspace(xmin, xmax, xdata.shape[0]), *linfitparams[0]
                ),
                mode="lines",
                name=p + "linear",
                line=dict(dash="dash", width=3, color=px.colors.qualitative.Plotly[i]),
            ),
            row=row,
            col=col,
        )

In [13]:
# from plotting import render_pain_current_subplot
# fig = go.Figure()
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Pain vs Current", f"Only >={pain_cutoff_for_linearfit}/10"),
    shared_xaxes=True,
    shared_yaxes=True,
    horizontal_spacing=0.05,
    x_title="Current (mA)",
    y_title="Pain report",
)

for i, p in enumerate(p_data.keys()):
    xdata = p_data[p]["data"]["mA"]
    ydata = p_data[p]["data"]["painreport"]
    linfitparams = p_data[p]["lfit"]
    expfitparams = p_data[p]["efit"]

    render_pain_current_subplot(
        xdata, ydata, expfitparams, linfitparams, p, fig, row=1, col=1
    )

    filtdat = p_data[p]["data"][
        p_data[p]["data"]["painreport"] >= pain_cutoff_for_linearfit
    ]

    render_pain_current_subplot(
        filtdat["mA"], filtdat["painreport"], expfitparams, linfitparams, p, fig, row=1, col=2
    )

fig.update_layout(
    title="Pain report vs. Current",
    font=dict(size=18),
    width=1000,
    height=500,
    template="plotly_dark",
    showlegend=False,
)

fig.update_yaxes(range=[0, 6])

fig.show()

In [14]:
fig = go.Figure()
for i, p in enumerate(p_data.keys()):
    xdata = p_data[p]["data"]["mA_norm"]
    ydata = p_data[p]["data"]["painreport"]
    linfitparams = p_data[p]["lfit_norm"]
    expfitparams = p_data[p]["efit_norm"]
    render_pain_current_plot(xdata, ydata, expfitparams, linfitparams, p, fig, plot_lin=False)

fig.update_layout(
    title="Pain report vs. Normalized current",
    xaxis_title="Current / Max current (mA_norm)",
    yaxis_title="Pain report",
    font=dict(size=18),
    width=800,
    height=500,
    template="plotly_dark",
    showlegend=False,
)
fig.update_yaxes(range=[-1, 6])

fig.show()